# Anvil Lite Cost Analysis

## Initial setup

Make sure to fill in `<REDACTED>` variables with their proper values.

In [ ]:
# You will need to reinstall the dependencies every time you restart the kernel. 
# Otherwise, you can skip this cell.

!git clone https://github.com/broadinstitute/anvil_lite_cost_analysis.git
!pip install ./anvil_lite_cost_analysis

In [ ]:
# You must import these dependencies every time the notebook is restarted

import os
import pandas as pd
from anvil_lite_cost_analysis import alca


In [ ]:
# Set variables. These will need updates when copying this notebook elsewhere!

config = alca.Config(
    wsm_url="https://workspace.dsde-prod.broadinstitute.org",
    rawls_url="https://rawls.dsde-prod.broadinstitute.org",
    current_workspace_id = os.environ['WORKSPACE_ID'],
    subscription_id = "<REDACTED>",
    storage_account = "<REDACTED>",
    target_mrgs = [
        "<REDACTED>", 
        "<REDACTED>_aks"
    ],
    container_name = os.environ['WORKSPACE_STORAGE_CONTAINER_URL'].split('/')[-1],
    container_id = os.environ['WORKSPACE_STORAGE_CONTAINER_ID'],
    blob_inventory_name = "AnvilLite_Azure_blob_inventory",
    blob_inventory_prefix = "2024",
    cost_management_storage_account = "costandstorageexports",
    cost_management_storage_container = "costanalysis",
    cost_management_directory = "landingzone",
    aks_management_storage_account = "costandstorageexports",
    aks_management_storage_container = "costanalysis",
    aks_management_directory = "aks",
    local_costs_url = "costexport/latest.csv",
    local_aks_costs_url = "costexport/latest-aks.csv",
    analysis_window_size = 30,
    cost_column_name = "PreTaxCost"
)

In [ ]:
####################################################################################################
####################################################################################################
####################################################################################################
#################### SECRETS #######################################################################
#################### remove these values from the notebook before sharing ##########################

# cost management account key
# TODO: figure out IAM-based auth instead!
cost_management_key = "<REDACTED>"

####################################################################################################
####################################################################################################
####################################################################################################

## Authorization

These two steps ensure you are authorized. You may need to re-run these steps if you have been working in the notebook for a significant amount of time and suddenly encounter permission problems.

In [ ]:
azure_token = alca.get_azure_token()

In [ ]:
sas_token = alca.get_sas_token(azure_token, config)

## Find the latest (and previous month's) cost exports

In [ ]:
# Retrieves the exports from Microsoft.
# You should run this each time the notebook is restarted.
exports = alca.get_exports(cost_management_key, config)

In [ ]:
# Copies the exports into this workspace's storage.
# You can skip this if you've already copied today's exports.
responses = alca.copy_exports_to_workspace(exports, sas_token, cost_management_key, config)

## Analysis

The remainder of steps in this notebook need to run for every analysis.

In [ ]:
analysis_data = alca.build_analysis_dataframes(exports, azure_token, sas_token, config)

storage_grouped = analysis_data.storage_grouped
costs = analysis_data.costs
costs_grouped = analysis_data.costs_grouped
costs_workspace_grouped = analysis_data.costs_workspace_grouped
costs_shared_grouped = analysis_data.costs_shared_grouped

In [ ]:
# display, using the human-readable string
display(storage_grouped[["workspace_or_container", "Total Size"]])

In [ ]:
# plot the storage inventory

# how many rows to show?
topN = 25

ax = storage_grouped[:topN].plot.barh(x="workspace_or_container", y="MB",
                                      logx=False, figsize=(7,7),
                                      xlabel = "MB",
                                      title="Top " + str(topN) + " workspaces or containers by size")
_ = ax.invert_yaxis()
# add labels to each bar
_ = ax.bar_label(ax.containers[0])


In [ ]:
# display shared costs
print("====== landing zone shared costs =====")
display(costs_shared_grouped[["MeterCategory", "Total Cost"]])
# display per-workspace costs
print("====== per-workspace costs =====")
display(costs_workspace_grouped[["workspace_name", "Total Cost"]])

# display all costs
print("====== all costs =====")
display(costs_grouped[["workspace_or_category", "Total Cost"]])

In [ ]:
# plot the per-workspace costs

# how many rows to show?
topNCost = 25

axc = costs_grouped[:topNCost].plot.barh(x="workspace_or_category", y=config.cost_column_name,
                                      logx=False, figsize=(7,7),
                                         xlabel="Total Cost",
                                      title="Top " + str(topNCost) + f" workspaces or services by cost, previous {config.analysis_window_size} days")
_ = axc.invert_yaxis()

# labels for bars
_ = axc.bar_label(axc.containers[0])



In [ ]:
# Enter the exact name of a cost category to use for drilldown:
# drilldown_cost_category = "AnVIL_Lite_Harvard_Carey_20240628"
drilldown_cost_category = "Virtual Machines (shared)"
# drilldown_cost_category = "da-costmanagement-01"


costs_drilldown = costs[costs["workspace_or_category"] == drilldown_cost_category].groupby(["workspace_or_category", "MeterCategory", "MeterSubcategory", "MeterName"], dropna=False)[config.cost_column_name].sum().to_frame().sort_values(by=config.cost_column_name, ascending=False).reset_index()
costs_drilldown["Total Cost"] = costs_drilldown[config.cost_column_name].map("${:,.2f}".format)
display(costs_drilldown[["workspace_or_category", "MeterCategory", "MeterSubcategory", "MeterName", "Total Cost"]])
